# Numerical solution of pipe problems

Solving for flow rate $Q$ in pipe problems is often tricky because pressure/heads are related to flow rate through complicated functions.
In the case of pipe losses, there is the dependence of friction factor on Reynold's number (the flow speed in Reynolds number is related to $Q/A$). 
This makes it difficult to analytically "move $Q$ to one side" of the equation and solve.

For practical problems you usually have to solve for $Q$ numerically.

A common way to do this is to rearrange the equation so that the solution is a 'root'. 
You might have learned strategies like this in your "SYDE 311: Advanced mathematics" course.
This is shown below.

In [ ]:
import numpy as np

In [ ]:
# Some constants we will use
G = 9.81

# Water properties
RHO = 1000
MU = 1e-3

## Solving for the friction factor

Solving for the friction factor requires numerical methods since you can't move 'f' over to one side.

Recall for turbulent flow, the friction factor is given by the Colebrook-white relation:
$$
\frac{1}{f^{1/2}} = -2\log(\frac{\epsilon/d}{3.7} + \frac{2.51}{\mathrm{Re}_\mathrm{d} f^{1/2}})
$$

To solve this numerically for a given $Re$ and $\epsilon/d$, we rearrange the equation:
$$
\frac{1}{f^{1/2}} + 2\log(\frac{\epsilon/d}{3.7} + \frac{2.51}{\mathrm{Re}_\mathrm{d} f^{1/2}}) = 0
$$
The 'root' of the left expression is the friction factor, $f$.

There are well-developed numerical methods for finding roots you can use to solve this (in matlab and python, for example).

Root methods often require an "initial guess" of the root to work. 
For this guess, we can use the strategy in class and assume $f$ is completely turbulent.

### Complete turbulence
Notice that for complete turbulence, $Re\to\infty$ and $\frac{2.51}{\mathrm{Re}_\mathrm{d} f^{1/2}} \to 0$. 
The colebrook-white relation is then 
$$
\frac{1}{f^{1/2}} + 2\log(\frac{\epsilon/d}{3.7}) = 0
$$
You can easily rearrange for $f$ and solve this!

In [ ]:
# This is the rearranged expression from above
def colebrook_white(friction_factor, reynolds_number, rel_roughness):
    """
    Return the "Colebrook-White" correlation residual
    """
    f = friction_factor
    re = reynolds_number
    rel_eps = rel_roughness
    return 1/f**0.5 + 2*np.log10(rel_eps/3.7 + 2.51/(re*f**0.5))

def colebrook_white_complete_turbulence(rel_roughness):
    """
    Return the "Colebrook-White" friction factor for complete turbulence

    Complete turbulence occurs as Reynold's number tends to infinity.
    In this case, the Colebrook-White equations can be explicitly solved for the friction factor
    , which depends only on relative roughness.
    """
    rel_eps = rel_roughness
    return (-2*np.log(rel_eps/3.7))**(-2)

In [ ]:
from scipy.optimize import root_scalar

# Imagine we have Re = 5000, and eps/d = 0.05
# Here's 
re = 5000
rel_eps = 0.05

# we can assume complete turbulence for out initial guess
f_complete = colebrook_white_complete_turbulence(rel_eps)
sol = root_scalar(colebrook_white, args=(re, rel_eps), x0=f_complete)
f = sol['root']

# Compare the solution you get here with the Moody chart!
print(f)

In [ ]:
# Use the strategy above to create a function that can return friction factors!   
def friction_factor(reynolds_number, rel_roughness):
    """
    Return the turbulent friction factor from the "Moody diagram"
    """
    re = reynolds_number
    rel_eps = rel_roughness
    
    f_complete = colebrook_white_complete_turbulence(rel_eps)
    sol = root_scalar(colebrook_white, args=(re, rel_eps), x0=f_complete)
    f = sol['root']
    return f

## Solving a pipe problem

Similar to the friction factor, solving the conservation of energy equation for $Q$ is difficult analytically.
We can use the same 'root' approach to solve these.

Imagine the simple pipe problem shown:

![title](SimplePipeProblem.png)

We know:
- inlet conditions $p_1=50 Pa, z_1 = 0$
- outlet conditions $p_2=0 Pa, z_2 = 0$
- pipe properties $d=10 cm, \epsilon=1 mm$


Conservation of energy for this problem should take the form
$$
(\frac{p_1}{\rho g} + \frac{v_1^2}{2g} + z_1) = (\frac{p_2}{\rho g} + \frac{v_2^2}{2g} + z_2) + h_f
$$
where $h_f$ is the head loss from the pipe.
Our goal is to find $Q$ from this.

In [ ]:
# The known inlet conditions at 1 are:
# 50 Pa, 1cm^2, 0 m elevation
# p1 = 50 Pa, 
p_1 = 50
area_1 = 1e-2 ** 2
z_1 = 0

def head_1(Q):
    # This is the "head" at 1
    # Note it depends on Q
    v = (Q/area_1)**2
    return p_1/(RHO*G) + (v)**2/(2*G) + z_1

In [ ]:
# The known outlet conditions at 2 are:
# 0 Pa, 1cm^2, 0 m elevation
# p1 = 50 Pa, 
p_2 = 00
area_2 = 1e-2 ** 2
z_2 = 0

def head_2(Q):
    # This is the "head" at 2
    # Note it depends on Q
    v = (Q/area_2)**2
    return p_2/(RHO*G) + (v)**2/(2*G) + z_2

In [ ]:
# Pipe head loss

# Pipe's relative roughness
eps = 1e-3
d_pipe = 10e-2
rel_eps = 1e-3/d_pipe
length = 1

area_pipe = np.pi*(d_pipe/2)**2

def head_f(Q):
    # This is the "head" loss over the pipe
    # Note it depends on Q
    v = (Q/area_pipe)**2
    re = RHO*v*d_pipe / MU

    # Note this comes from our Colebrook-white relation
    f = friction_factor(re, rel_eps)
    return length/d_pipe * v**2/(2*G)*f

In [ ]:
# The energy equations says:
def energy_equation(Q):
    return head_1(Q) - head_2(Q) - head_f(Q)

# Here we guess Q = 1m^3/s as the initial guess
# There's probably much better ways to guess Q! (e.g. assume no head losses)
sol = root_scalar(energy_equation, x0=1)
print(sol)
print(sol['root'])